## Import Dependencies

In [1]:
# importing modules
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from pprint import pprint


## Read athlete height dataset
### Source
enter source url here

### About
Contents: all Winter/Summer Olympic athletes and the events they participated in between 1898 and 2016<br>
Size: The raw data set is 271,116 rows

In [2]:
# athlete events df from csv
athletes_file = os.path.join("..","data", "rawData","athlete_events.csv")
athletes_df = pd.read_csv(athletes_file, encoding="ISO-8859-1")

# To get the raw data count, commented out to keep the original df .head() results prominent
raw_count = len(athletes_df)

# preview the raw data
athletes_df.head()

ID                      Name Sex   Age  Height  Weight            Team  \
0   1                 A Dijiang   M  24.0   180.0    80.0           China   
1   2                  A Lamusi   M  23.0   170.0    60.0           China   
2   3       Gunnar Nielsen Aaby   M  24.0     NaN     NaN         Denmark   
3   4      Edgar Lindenau Aabye   M  34.0     NaN     NaN  Denmark/Sweden   
4   5  Christine Jacoba Aaftink   F  21.0   185.0    82.0     Netherlands   

   NOC        Games  Year  Season       City          Sport  \
0  CHN  1992 Summer  1992  Summer  Barcelona     Basketball   
1  CHN  2012 Summer  2012  Summer     London           Judo   
2  DEN  1920 Summer  1920  Summer  Antwerpen       Football   
3  DEN  1900 Summer  1900  Summer      Paris     Tug-Of-War   
4  NED  1988 Winter  1988  Winter    Calgary  Speed Skating   

                              Event Medal  
0       Basketball Men's Basketball   NaN  
1      Judo Men's Extra-Lightweight   NaN  
2           Football Men's Football   NaN  
3       Tug-Of-War Men's Tug-Of-War  Gold  
4  Speed Skating Women's 500 metres   NaN

## Winter, Summer, or Both?
### Description:
Our first analysis centered around the Season of the games in our data.  We batted ideas around like, it's possible that there is a bias in Winter sports because not every country has the opportunity/capitol/etc.  But before we started down that path, we looked to see how much data we had of each:

Summer: 222,552 (82%)
Winter:  48,564 (18%)


In [3]:
# checking to see how much data we have for each season
season_df = pd.DataFrame(athletes_df.groupby('Season')['ID'].count())

# adding a column for overall percentage
season_df['Percent of Total'] = (100*(season_df['ID']/raw_count)).round(1)

# look at %'s'
season_df.head()

# Should we plot this?

ID  Percent of Total
Season                          
Summer  222552              82.1
Winter   48564              17.9

## Goodbye Snow/Ice!
### Removing Winter from our data
Since we had so much more summer data, plus other factors (Winter games used to share the same 4 year cycle, now the alternate...blah...blah), we decided to remove the Winter games from our data.  The new summer only data frame has 222,552 rows.


In [4]:
# New data fram to store the summer data only, using "loc"
summer_df = athletes_df.loc[athletes_df['Season']=='Summer']

# making sure it's a datafram (spoiler alert, it is)
# type(summer_df)

# get length of summer, just to verify
summer_count = len(summer_df)

# print once just to verify
# print(summer_count)

# Preview of Summer data
summer_df.head()

ID                                Name Sex   Age  Height  Weight  \
0    1                           A Dijiang   M  24.0   180.0    80.0   
1    2                            A Lamusi   M  23.0   170.0    60.0   
2    3                 Gunnar Nielsen Aaby   M  24.0     NaN     NaN   
3    4                Edgar Lindenau Aabye   M  34.0     NaN     NaN   
26   8  Cornelia "Cor" Aalten (-Strannood)   F  18.0   168.0     NaN   

              Team  NOC        Games  Year  Season         City       Sport  \
0            China  CHN  1992 Summer  1992  Summer    Barcelona  Basketball   
1            China  CHN  2012 Summer  2012  Summer       London        Judo   
2          Denmark  DEN  1920 Summer  1920  Summer    Antwerpen    Football   
3   Denmark/Sweden  DEN  1900 Summer  1900  Summer        Paris  Tug-Of-War   
26     Netherlands  NED  1932 Summer  1932  Summer  Los Angeles   Athletics   

                           Event Medal  
0    Basketball Men's Basketball   NaN  
1   Judo Men's Extra-Lightweight   NaN  
2        Football Men's Football   NaN  
3    Tug-Of-War Men's Tug-Of-War  Gold  
26  Athletics Women's 100 metres   NaN

## Time to delete all NaN, or Null, values without any investigation (just kidding Chris!)
### Missing Heights
We noticed right away that some of the height values were missing, particularly for the older Olympic games.  

### Find rows where athlete height are missing
We found 51,857 rows, or 23% of our Summer data, with missing height values.

In [5]:
# looking at records missing height
missing_df = summer_df.loc[summer_df['Height'].isnull()]
# missing_df.head()

# variable for missing count and percentage
missing_count = len(missing_df)
missing_pct = (100*(int(missing_count) / int(summer_count)))

# having trouble rounding that float!

# print the results...
print(f'Number of athlete/event rows missing the "height" value: {missing_count} {missing_pct}%')

Number of athlete/event rows missing the "height" value: 51857 23.30107121032388%


## Additional validation
Making sure the null count total made sense, by verifying with a group by year, and then sum... (it checks out!)

In [6]:
# create a group by object to count the number of missing rows by year
nullYear_group = missing_df.groupby('Year')['ID'].count()

nullYear_sum = nullYear_group.sum()

# print results
print(f'There is still this many rows missing height: {nullYear_sum}')


There is still this many rows missing height: 51857


## See how much this impacts us per Olympic year
We then looked to see how that 23% missing data impacted us by year.

In [7]:
# make a new df, grouping the summer data by year, counting the ID
year_count = pd.DataFrame(summer_df.groupby('Year')['ID'].count())

# add a new column to show missing count
year_count['Missing Height'] = nullYear_group

# add a new column to show percentage
year_count['Missing Pct'] = (100*(nullYear_group / year_count['ID'])).round(2)

# show all years, and their percent
year_count

# should we chart this? (bar chart by year, add label with the % missing?)


ID  Missing Height  Missing Pct
Year                                    
1896    380             334        87.89
1900   1936            1820        94.01
1904   1301            1088        83.63
1906   1733            1476        85.17
1908   3101            2626        84.68
1912   4040            3319        82.15
1920   4292            3525        82.13
1924   5233            4348        83.09
1928   4992            4139        82.91
1932   2969            1952        65.75
1936   6506            5450        83.77
1948   6405            5352        83.56
1952   8270            6232        75.36
1956   5127            2785        54.32
1960   8119             381         4.69
1964   7702             272         3.53
1968   8588              95         1.11
1972  10304             286         2.78
1976   8641             358         4.14
1980   7191             230         3.20
1984   9454             422         4.46
1988  12037             318         2.64
1992  12977            2524        19.45
1996  13780            1871        13.58
2000  13821             123         0.89
2004  13443              36         0.27
2008  13602             151         1.11
2012  12920             168         1.30
2016  13688             176         1.29

## Going post-war, hippie era!
### Focusing on the Summer games from 1960-2016
We decided since all of the games before 1960 were missing at least 50% of their height data, we decided to focus on the 15 games between 1960 and 2016.  We are now down to just 166,267 rows, we hope it's enough!

In [8]:
# New df of just post 60s Summer games
summerModern_df = summer_df.loc[summer_df['Year']>=1960]

# grab the count
summerModern_count = len(summerModern_df)

# we already have view of the same data above, so just print the new row count for now
print(f'In the modern era Summer games data, there are {summerModern_count} rows')

In the modern era Summer games data, there are 166267 rows


## Will still have null heights
Right now, we are dropping the remainder.<br>The new row count is 158,856.<br><br>We should maybe revisit and see if the missing data is skewed towards certain countries

In [9]:
# the dreaded drop nulls call...
summerClean_df = summerModern_df.dropna(how='any',subset=['Height'])

# get the count
summerClean_count = len(summerClean_df)

# preview the new df
summerClean_df.head()

# print the row count
print(f'There are {summerClean_count} rows left!')

There are 158856 rows left!


## Converting that crazy metric stuff...
### replacing the metric height and weight with the correct values!  Also adding new columns for medaling, yes or now
Cleaning the data, and adding a new column for categorical analysis


In [10]:
# converting metric to the correct system
type(summerClean_df)

summerClean_df['Height'] = summerClean_df['Height'].apply(lambda x: x / 2.54)
summerClean_df['Weight'] = summerClean_df['Weight'].apply(lambda x: x / .453592)

# Adding a new column based on whether or not the athlete medaled in the event
summerClean_df.loc[summerClean_df.Medal.isnull(), 'MedalYN']='No'
summerClean_df.loc[summerClean_df.Medal.notnull(), 'MedalYN']='Yes'

# adding a new sub-event column to get the specific event (i.e. "shotput")


# # preview
summerClean_df.head()


/Users/kasiewong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/kasiewong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/kasiewong/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

ID                Name Sex   Age     Height      Weight     Team  NOC  \
0    1           A Dijiang   M  24.0  70.866142  176.369954    China  CHN   
1    2            A Lamusi   M  23.0  66.929134  132.277465    China  CHN   
31  12   Jyri Tapani Aalto   M  31.0  67.716535  154.323709  Finland  FIN   
32  13  Minna Maarit Aalto   F  30.0  62.598425  122.356655  Finland  FIN   
33  13  Minna Maarit Aalto   F  34.0  62.598425  122.356655  Finland  FIN   

          Games  Year  Season       City       Sport  \
0   1992 Summer  1992  Summer  Barcelona  Basketball   
1   2012 Summer  2012  Summer     London        Judo   
31  2000 Summer  2000  Summer     Sydney   Badminton   
32  1996 Summer  1996  Summer    Atlanta     Sailing   
33  2000 Summer  2000  Summer     Sydney     Sailing   

                           Event Medal MedalYN  
0    Basketball Men's Basketball   NaN      No  
1   Judo Men's Extra-Lightweight   NaN      No  
31       Badminton Men's Singles   NaN      No  
32    Sailing Women's Windsurfer   NaN      No  
33    Sailing Women's Windsurfer   NaN      No

## How many sports per year, in our 15 year set?
### We thought it would be prudent to check and see how many sports we have per year.
Out of 36 sports in the Summer Games since 1960, 19 (53%) have data in all 15 Olympic games.<br>Do we need a chart here?

In [11]:
# sports = df['Sport'].unique().tolist()

# data frame of unique sports and year combinations
sportYears_df = pd.DataFrame(summerClean_df.groupby(['Sport','Year'])['Year'].count())

# another group by, this time to count the years for each sport (to make distribution)
sport_df = pd.DataFrame(sportYears_df.groupby('Sport')['Year'].count().reset_index())

# get the number of sports
sport_count = len(sport_df)

# see how many of them have 15 years
sportFull_df = sport_df.loc[sport_df['Year']==15]

# get the count of sports that have the full 15 Olympics
sportFull_count = len(sportFull_df)
sportFull_pct = 100*(sportFull_count / sport_count)

# print the results
print(f'Out of {sport_count} sports, only {sportFull_count}, or {sportFull_pct}% have 15 Olymic games data.')


Out of 36 sports, only 19, or 52.77777777777778% have 15 Olymic games data.


## Zoom in on the 19 sports with data in all 15 games
### more changes to the summer clean data
Cutting down to 19 sports<br>
There are 136,692 rows left to work with...

In [12]:
# drop the year column from our keep sports dataframe, we only need the sport
sportFull_df = sportFull_df.drop('Year', axis=1)

# merge the summer clean data with the sports list df, this will cut it to 19 sports
summerClean_df = pd.merge(sportFull_df, summerClean_df, on='Sport', how='inner')

# get the count
summerClean_count = len(summerClean_df)

# print the results
print(f'There are now {summerClean_count} rows to work with!')
      
# preview the data      
# summerClean_df.head()


There are now 136692 rows to work with!


## splitting out the data sets by gender
We need to fill in why, but we are splitting up the genders...<br>
There are 42,278 rows for women and 94,414 rows for men.

In [13]:
# olympic data for women
womens_data = summerClean_df.loc[summerClean_df['Sex']=='F']

# olympic data for men only...
mens_data = summerClean_df.loc[summerClean_df['Sex']=='M']

# get the count of data sets
mens_count = len(mens_data)
womens_count = len(womens_data)

print(f'There are {womens_count} rows for women and {mens_count} for men.')

There are 42278 rows for women and 94414 for men.


## Removed "Mixed" Sub events

In [14]:
mens_nonmixed_data = mens_data.loc[~mens_data['Event'].str.contains("Mixed")]
mens_nonmixed_data

Sport      ID                                       Name Sex  \
0       Athletics      18                       Timo Antero Aaltonen   M   
1       Athletics      34         Jamale (Djamel-) Aarrass (Ahrass-)   M   
4       Athletics      52                              Patrick Abada   M   
5       Athletics      55                        Antonio Abadia Beci   M   
8       Athletics      86                    Jos Manuel Abascal Gmez   M   
9       Athletics      86                    Jos Manuel Abascal Gmez   M   
11      Athletics      91                             Emanuele Abate   M   
13      Athletics      96          Carlos Rodolfo Abaunza Balladares   M   
14      Athletics      97                     Carlos Alberto Abaunza   M   
15      Athletics     109  Abdul Latif Al-Sayed Abbas Youssef Hashem   M   
16      Athletics     109  Abdul Latif Al-Sayed Abbas Youssef Hashem   M   
17      Athletics     109  Abdul Latif Al-Sayed Abbas Youssef Hashem   M   
18      Athletics     110                       Abubakar Abbas Abbas   M   
19      Athletics     111                               Aqarab Abbas   M   
20      Athletics     114                    Essa Abbas Faleh Hashem   M   
21      Athletics     115                               Ghulam Abbas   M   
22      Athletics     117                                Maher Abbas   M   
23      Athletics     117                                Maher Abbas   M   
24      Athletics     129                             Ruslan Abbasov   M   
25      Athletics     160                        Ali Abdalla Afringi   M   
26      Athletics     166                             Omari Abdallah   M   
27      Athletics     166                             Omari Abdallah   M   
28      Athletics     168                      Younis Abdallah Rabee   M   
30      Athletics     179                   Ibrahim Saad Abdel Galil   M   
31      Athletics     179                   Ibrahim Saad Abdel Galil   M   
32      Athletics     194                Moustafa Mounib Abdel Kader   M   
33      Athletics     204                       Moustafa Abdel Naser   M   
34      Athletics     234               Abdalla Abdelgadir El-Sheikh   M   
35      Athletics     243      Ihab Abdelrahman El-Sayed Abdelrahman   M   
36      Athletics     247                        El-Hachemi Abdenouz   M   
...           ...     ...                                        ...  ..   
136651  Wrestling  134394                            Zevegiin Dvchin   M   
136652  Wrestling  134394                            Zevegiin Dvchin   M   
136653  Wrestling  134394                            Zevegiin Dvchin   M   
136654  Wrestling  134395                             Zevegiin Oidov   M   
136655  Wrestling  134395                             Zevegiin Oidov   M   
136656  Wrestling  134395                             Zevegiin Oidov   M   
136657  Wrestling  134408           Elmadi Zaynaydiyevich Zhabrailov   M   
136658  Wrestling  134408           Elmadi Zaynaydiyevich Zhabrailov   M   
136659  Wrestling  134435                             Zhang Chongyao   M   
136660  Wrestling  134441                                Zhang Dequn   M   
136663  Wrestling  134640                                Zhao Hailin   M   
136664  Wrestling  134644                             Zhao Jianqiang   M   
136666  Wrestling  134707                              Doncho Zhekov   M   
136667  Wrestling  134716                   Valentin Dimitrov Zhelev   M   
136672  Wrestling  134878                      Vitaly Ivanovich Zhuk   M   
136673  Wrestling  134891         Dauren Nurdauletovich Zhumagaziyev   M   
136674  Wrestling  134989                            David Zilberman   M   
136675  Wrestling  135141                                 Emin zizov   M   
136679  Wrestling  135174                                 Gad Zobari   M   
136680  Wrestling  135188                         Andr Georges Zoete   M   
136681  Wrestling  135188                         Andr Georg

## Break out Sub Events

In [15]:
event_group = pd.DataFrame(mens_nonmixed_data.groupby("Event")["ID"].count().reset_index())
event_group

Event    ID
0                      Athletics Men's 1,500 metres   697
1                     Athletics Men's 10,000 metres   584
2                        Athletics Men's 100 metres  1152
3                Athletics Men's 110 metres Hurdles   574
4                Athletics Men's 20 kilometres Walk   659
5                        Athletics Men's 200 metres   909
6         Athletics Men's 3,000 metres Steeplechase   536
7              Athletics Men's 4 x 100 metres Relay  1375
8              Athletics Men's 4 x 400 metres Relay  1299
9                        Athletics Men's 400 metres   867
10               Athletics Men's 400 metres Hurdles   581
11                     Athletics Men's 5,000 metres   654
12               Athletics Men's 50 kilometres Walk   649
13                       Athletics Men's 800 metres   815
14                        Athletics Men's Decathlon   486
15                     Athletics Men's Discus Throw   485
16                     Athletics Men's Hammer Throw   443
17                        Athletics Men's High Jump   529
18                    Athletics Men's Javelin Throw   462
19                        Athletics Men's Long Jump   585
20                         Athletics Men's Marathon  1390
21                       Athletics Men's Pole Vault   435
22                         Athletics Men's Shot Put   427
23                      Athletics Men's Triple Jump   546
24                      Basketball Men's Basketball  2322
25                        Boxing Men's Bantamweight   451
26                       Boxing Men's Featherweight   407
27                           Boxing Men's Flyweight   414
28                         Boxing Men's Heavyweight   241
29                     Boxing Men's Light-Flyweight   328
..                                              ...   ...
139               Weightlifting Men's Featherweight   276
140                   Weightlifting Men's Flyweight   125
141                 Weightlifting Men's Heavyweight   196
142               Weightlifting Men's Heavyweight I    79
143              Weightlifting Men's Heavyweight II    75
144           Weightlifting Men's Light-Heavyweight   296
145                 Weightlifting Men's Lightweight   308
146          Weightlifting Men's Middle-Heavyweight   305
147                Weightlifting Men's Middleweight   281
148           Weightlifting Men's Super-Heavyweight   183
149         Wrestling Men's Bantamweight, Freestyle   212
150       Wrestling Men's Bantamweight, Greco-Roman   218
151        Wrestling Men's Featherweight, Freestyle   302
152      Wrestling Men's Featherweight, Greco-Roman   286
153            Wrestling Men's Flyweight, Freestyle   194
154          Wrestling Men's Flyweight, Greco-Roman   169
155          Wrestling Men's Heavyweight, Freestyle   248
156        Wrestling Men's Heavyweight, Greco-Roman   231
157      Wrestling Men's Light-Flyweight, Freestyle    96
158    Wrestling Men's Light-Flyweight, Greco-Roman   103
159    Wrestling Men's Light-Heavyweight, Freestyle   280
160  Wrestling Men's Light-Heavyweight, Greco-Roman   265
161          Wrestling Men's Lightweight, Freestyle   300
162        Wrestling Men's Lightweight, Greco-Roman   285
163         Wrestling Men's Middleweight, Freestyle   292
164       Wrestling Men's Middleweight, Greco-Roman   277
165    Wrestling Men's Super-Heavyweight, Freestyle   189
166  Wrestling Men's Super-Heavyweight, Greco-Roman   188
167         Wrestling Men's Welterweight, Freestyle   309
168       Wrestling Men's Welterweight, Greco-Roman   286

[169 rows x 2 columns]

## Clean up Sub Events

In [16]:
subevents_data = mens_nonmixed_data["Event"].replace("Men's ","",regex=True)
subevents_data
mens_nonmixed_data["Sub Event"]=subevents_data
mens_nonmixed_data.head()

/Users/kasiewong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Sport  ID                                Name Sex   Age     Height  \
0  Athletics  18                Timo Antero Aaltonen   M  31.0  74.409449   
1  Athletics  34  Jamale (Djamel-) Aarrass (Ahrass-)   M  30.0  73.622047   
4  Athletics  52                       Patrick Abada   M  22.0  74.409449   
5  Athletics  55                 Antonio Abadia Beci   M  26.0  66.929134   
8  Athletics  86             Jos Manuel Abascal Gmez   M  22.0  71.653543   

       Weight     Team  NOC        Games  Year  Season            City  \
0  286.601175  Finland  FIN  2000 Summer  2000  Summer          Sydney   
1  167.551456   France  FRA  2012 Summer  2012  Summer          London   
4  176.369954   France  FRA  1976 Summer  1976  Summer        Montreal   
5  143.300587    Spain  ESP  2016 Summer  2016  Summer  Rio de Janeiro   
8  147.709836    Spain  ESP  1980 Summer  1980  Summer          Moskva   

                          Event Medal MedalYN               Sub Event  
0      Athletics Men's Shot Put   NaN      No      Athletics Shot Put  
1  Athletics Men's 1,500 metres   NaN      No  Athletics 1,500 metres  
4    Athletics Men's Pole Vault   NaN      No    Athletics Pole Vault  
5  Athletics Men's 5,000 metres   NaN      No  Athletics 5,000 metres  
8  Athletics Men's 1,500 metres   NaN      No  Athletics 1,500 metres

   ## Find Average Heights of Olympians by Year and Sub Event

In [56]:
mens_meanheight_data=list(mens_nonmixed_data.groupby(['Year','Sub Event'])['Height'].mean())
mens_minheight_data=list(mens_nonmixed_data.groupby(['Year','Sub Event'])['Height'].min())
mens_maxheight_data=list(mens_nonmixed_data.groupby(['Year','Sub Event'])['Height'].max())
mens_medianheight_data=list(mens_nonmixed_data.groupby(['Year','Sub Event'])['Height'].median())
mens_meanheight_data
mens_minheight_data
mens_maxheight_data
mens_medianheight_data


[70.07874015748031,
 67.71653543307086,
 69.29133858267717,
 72.24409448818898,
 70.07874015748031,
 70.07874015748031,
 68.89763779527559,
 68.89763779527559,
 70.07874015748031,
 69.88188976377953,
 70.47244094488188,
 67.71653543307086,
 69.68503937007874,
 70.07874015748031,
 72.44094488188976,
 74.01574803149606,
 72.24409448818898,
 73.22834645669292,
 72.24409448818898,
 70.86614173228347,
 67.32283464566929,
 70.86614173228347,
 74.40944881889763,
 70.86614173228347,
 74.40944881889763,
 64.1732283464567,
 66.14173228346456,
 63.38582677165354,
 72.83464566929133,
 71.25984251968504,
 69.29133858267717,
 68.11023622047244,
 66.53543307086615,
 69.29133858267717,
 67.71653543307086,
 68.89763779527559,
 71.65354330708661,
 70.07874015748031,
 69.68503937007874,
 69.88188976377953,
 69.68503937007874,
 68.89763779527559,
 68.50393700787401,
 68.89763779527559,
 68.11023622047244,
 69.68503937007874,
 66.92913385826772,
 67.51968503937007,
 68.7007874015748,
 68.50393700787401,
 6

## Create DataFrame with Height Calcs

In [58]:
agg_df=pd.DataFrame(mens_nonmixed_data.groupby(['Year','Sub Event'])['ID'].count().reset_index())
agg_df['Average Height']=mens_meanheight_data
agg_df['Median Height']=mens_medianheight_data
agg_df['Min Height']=mens_minheight_data
agg_df['Max Height']=mens_maxheight_data

agg_df

Year                                 Sub Event   ID  Average Height  \
0     1960                    Athletics 1,500 metres   39       69.715324   
1     1960                   Athletics 10,000 metres   32       68.048720   
2     1960                      Athletics 100 metres   59       69.331376   
3     1960              Athletics 110 metres Hurdles   36       72.561242   
4     1960              Athletics 20 kilometres Walk   36       69.619423   
5     1960                      Athletics 200 metres   61       69.865754   
6     1960       Athletics 3,000 metres Steeplechase   32       68.996063   
7     1960            Athletics 4 x 100 metres Relay   77       69.368034   
8     1960            Athletics 4 x 400 metres Relay   76       70.348114   
9     1960                      Athletics 400 metres   54       69.903762   
10    1960              Athletics 400 metres Hurdles   34       70.368226   
11    1960                    Athletics 5,000 metres   47       68.009717   
12    1960              Athletics 50 kilometres Walk   39       69.715324   
13    1960                      Athletics 800 metres   50       70.070866   
14    1960                       Athletics Decathlon   30       72.467192   
15    1960                    Athletics Discus Throw   35       73.858268   
16    1960                    Athletics Hammer Throw   28       72.300337   
17    1960                       Athletics High Jump   32       73.056102   
18    1960                   Athletics Javelin Throw   28       71.976940   
19    1960                       Athletics Long Jump   48       70.685696   
20    1960                        Athletics Marathon   69       67.106014   
21    1960                      Athletics Pole Vault   29       71.191963   
22    1960                        Athletics Shot Put   24       74.179790   
23    1960                     Athletics Triple Jump   39       70.795477   
24    1960                     Basketball Basketball  192       74.140830   
25    1960                       Boxing Bantamweight   29       64.376867   
26    1960                      Boxing Featherweight   30       66.062992   
27    1960                          Boxing Flyweight   31       63.144526   
28    1960                        Boxing Heavyweight   17       72.788328   
29    1960                  Boxing Light-Heavyweight   19       71.881475   
...    ...                                       ...  ...             ...   
1930  2016             Swimming 200 metres Butterfly   29       72.739614   
1931  2016             Swimming 200 metres Freestyle   46       74.580623   
1932  2016     Swimming 200 metres Individual Medley   27       73.680373   
1933  2016   Swimming 4 x 100 metres Freestyle Relay   72       75.256999   
1934  2016      Swimming 4 x 100 metres Medley Relay   74       74.404129   
1935  2016   Swimming 4 x 200 metres Freestyle Relay   72       74.360236   
1936  2016             Swimming 400 metres Freestyle   49       73.316728   
1937  2016     Swimming 400 metres Individual Medley   27       72.965879   
1938  2016              Swimming 50 metres Freestyle   74       73.499681   
1939  2016                     Water Polo Water Polo  154       75.340014   
1940  2016                Weightlifting Bantamweight   19       61.583092   
1941  2016               Weightlifting Featherweight   17       62.760537   
1942  2016                 Weightlifting Heavyweight   17       70.819824   
1943  2016           Weightlifting Light-Heavyweight   23       67.802123   
1944  2016                 Weightlifting Lightweight   21       64.866892   
1945  2016          Weightlifting Middle-Heavyweight   18       68.569554   
1946  2016                Weightlifting Middleweight   14       67.238470   
1947  2016           Weightlifting Super-Heavyweight   23       73.296816   
1948  2016        Wrestling Featherweight, Freestyle   20       64.232283   
1949  2016      Wrestling Featherweight, Greco-Roman   19       64.815582   
19

## Removing Sub Events that don't appear in every Olympiad

In [59]:
mens_nonmixed_data


Sport      ID                                       Name Sex  \
0       Athletics      18                       Timo Antero Aaltonen   M   
1       Athletics      34         Jamale (Djamel-) Aarrass (Ahrass-)   M   
4       Athletics      52                              Patrick Abada   M   
5       Athletics      55                        Antonio Abadia Beci   M   
8       Athletics      86                    Jos Manuel Abascal Gmez   M   
9       Athletics      86                    Jos Manuel Abascal Gmez   M   
11      Athletics      91                             Emanuele Abate   M   
13      Athletics      96          Carlos Rodolfo Abaunza Balladares   M   
14      Athletics      97                     Carlos Alberto Abaunza   M   
15      Athletics     109  Abdul Latif Al-Sayed Abbas Youssef Hashem   M   
16      Athletics     109  Abdul Latif Al-Sayed Abbas Youssef Hashem   M   
17      Athletics     109  Abdul Latif Al-Sayed Abbas Youssef Hashem   M   
18      Athletics     110                       Abubakar Abbas Abbas   M   
19      Athletics     111                               Aqarab Abbas   M   
20      Athletics     114                    Essa Abbas Faleh Hashem   M   
21      Athletics     115                               Ghulam Abbas   M   
22      Athletics     117                                Maher Abbas   M   
23      Athletics     117                                Maher Abbas   M   
24      Athletics     129                             Ruslan Abbasov   M   
25      Athletics     160                        Ali Abdalla Afringi   M   
26      Athletics     166                             Omari Abdallah   M   
27      Athletics     166                             Omari Abdallah   M   
28      Athletics     168                      Younis Abdallah Rabee   M   
30      Athletics     179                   Ibrahim Saad Abdel Galil   M   
31      Athletics     179                   Ibrahim Saad Abdel Galil   M   
32      Athletics     194                Moustafa Mounib Abdel Kader   M   
33      Athletics     204                       Moustafa Abdel Naser   M   
34      Athletics     234               Abdalla Abdelgadir El-Sheikh   M   
35      Athletics     243      Ihab Abdelrahman El-Sayed Abdelrahman   M   
36      Athletics     247                        El-Hachemi Abdenouz   M   
...           ...     ...                                        ...  ..   
136651  Wrestling  134394                            Zevegiin Dvchin   M   
136652  Wrestling  134394                            Zevegiin Dvchin   M   
136653  Wrestling  134394                            Zevegiin Dvchin   M   
136654  Wrestling  134395                             Zevegiin Oidov   M   
136655  Wrestling  134395                             Zevegiin Oidov   M   
136656  Wrestling  134395                             Zevegiin Oidov   M   
136657  Wrestling  134408           Elmadi Zaynaydiyevich Zhabrailov   M   
136658  Wrestling  134408           Elmadi Zaynaydiyevich Zhabrailov   M   
136659  Wrestling  134435                             Zhang Chongyao   M   
136660  Wrestling  134441                                Zhang Dequn   M   
136663  Wrestling  134640                                Zhao Hailin   M   
136664  Wrestling  134644                             Zhao Jianqiang   M   
136666  Wrestling  134707                              Doncho Zhekov   M   
136667  Wrestling  134716                   Valentin Dimitrov Zhelev   M   
136672  Wrestling  134878                      Vitaly Ivanovich Zhuk   M   
136673  Wrestling  134891         Dauren Nurdauletovich Zhumagaziyev   M   
136674  Wrestling  134989                            David Zilberman   M   
136675  Wrestling  135141                                 Emin zizov   M   
136679  Wrestling  135174                                 Gad Zobari   M   
136680  Wrestling  135188                         Andr Georges Zoete   M   
136681  Wrestling  135188                         Andr Georg

In [66]:


# data frame of unique sports and year combinations
SubEventyears_df = pd.DataFrame(mens_nonmixed_data.groupby(['Sub Event','Year'])['ID'].count().reset_index())
# SubEventyears_df.head()
# another group by, this time to count the years for each sport (to make distribution)
SubEvent_df = pd.DataFrame(SubEventyears_df.groupby('Sub Event')['ID'].count().reset_index())
# SubEvent_df
# # get the number of sports
SubEvent_count = len(SubEvent_df)

# # see how many of them have 15 years

SubEventFull_df = SubEvent_df.loc[SubEvent_df['ID']==15]
SubEventFull_df

SubEventFull_df = SubEventFull_df.drop('ID', axis=1)
further_cleaned_data = pd.merge(SubEventFull_df, agg_df, on='Sub Event', how='inner')
further_cleaned_data

Sub Event  Year  ID  Average Height  \
0                  Athletics 1,500 metres  1960  39       69.715324   
1                  Athletics 1,500 metres  1964  43       70.069584   
2                  Athletics 1,500 metres  1968  54       70.158938   
3                  Athletics 1,500 metres  1972  65       70.290733   
4                  Athletics 1,500 metres  1976  40       70.649606   
5                  Athletics 1,500 metres  1980  37       70.078740   
6                  Athletics 1,500 metres  1984  57       69.919878   
7                  Athletics 1,500 metres  1988  57       70.258323   
8                  Athletics 1,500 metres  1992  47       69.886078   
9                  Athletics 1,500 metres  1996  55       70.586972   
10                 Athletics 1,500 metres  2000  40       70.167323   
11                 Athletics 1,500 metres  2004  38       70.555325   
12                 Athletics 1,500 metres  2008  46       69.394043   
13                 Athletics 1,500 metres  2012  39       70.371492   
14                 Athletics 1,500 metres  2016  40       70.324803   
15                Athletics 10,000 metres  1960  32       68.048720   
16                Athletics 10,000 metres  1964  38       68.462495   
17                Athletics 10,000 metres  1968  37       68.503937   
18                Athletics 10,000 metres  1972  52       67.860388   
19                Athletics 10,000 metres  1976  41       69.041675   
20                Athletics 10,000 metres  1980  38       67.654372   
21                Athletics 10,000 metres  1984  44       68.423407   
22                Athletics 10,000 metres  1988  51       67.979003   
23                Athletics 10,000 metres  1992  53       67.657109   
24                Athletics 10,000 metres  1996  45       68.582677   
25                Athletics 10,000 metres  2000  34       68.515516   
26                Athletics 10,000 metres  2004  24       66.748688   
27                Athletics 10,000 metres  2008  36       66.622922   
28                Athletics 10,000 metres  2012  29       68.680424   
29                Athletics 10,000 metres  2016  30       68.044619   
...                                   ...   ...  ..             ...   
1230    Wrestling Welterweight, Freestyle  1960  22       67.859699   
1231    Wrestling Welterweight, Freestyle  1964  21       68.447694   
1232    Wrestling Welterweight, Freestyle  1968  19       69.249896   
1233    Wrestling Welterweight, Freestyle  1972  24       68.503937   
1234    Wrestling Welterweight, Freestyle  1976  21       69.047619   
1235    Wrestling Welterweight, Freestyle  1980  17       68.017601   
1236    Wrestling Welterweight, Freestyle  1984  22       68.700787   
1237    Wrestling Welterweight, Freestyle  1988  28       68.138358   
1238    Wrestling Welterweight, Freestyle  1992  14       69.038245   
1239    Wrestling Welterweight, Freestyle  1996  21       69.328834   
1240    Wrestling Welterweight, Freestyle  2000  20       66.988189   
1241    Wrestling Welterweight, Freestyle  2004  21       67.172853   
1242    Wrestling Welterweight, Freestyle  2008  21       66.985377   
1243    Wrestling Welterweight, Freestyle  2012  17       67.091246   
1244    Wrestling Welterweight, Freestyle  2016  21       66.347957   
1245  Wrestling Welterweight, Greco-Roman  1960  26       68.337371   
1246  Wrestling Welterweight, Greco-Roman  1964  19       68.441774   
1247  Wrestling Welterweight, Greco-Roman  1968  21       69.160105   
1248  Wrestling Welterweight, Greco-Roman  1972  19       68.586821   
1249  Wrestling Welterweight, Greco-Roman  1976  18       68.438320   
1250  Wrestling Welterweight, Greco-Roman  1980  13       68.503937   
1251  Wrestling Welterweight, Greco-Roman  1984  17       68.457619   
1252  Wrestling Welterweight, Greco-Roman  1988  19       68.876917   
1253  Wrestling Welterweight, Greco-Roman  1992  20       68.366142   
1254  Wrestling Welterweight, Greco-Roman  1996  18     

## Plot agg data to look for trends

In [68]:
y=further_cleaned_data['Median Height']
# type(y)
x=further_cleaned_data['Year']
# type(x)

plt.plot(x,y)

## Importing the country olympic country code data...

In [ ]:
# noc regions
regions_file = os.path.join("..", "data", "rawData", "noc_regions.csv")
regions_df = pd.read_csv(regions_file, encoding="ISO-8859-1")
regions_df.head()